In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder

import math

import tensorflow as tf

from tensorflow import keras 

from keras import layers, optimizers


from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import mglearn
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt   

Convert all categorical features' values into integers

In [2]:
#pass the train df into this function 
def labelconvert(df):
  df=df.drop(['Label','Section','POS','TAG','DEP','Shape','Is_Alpha','Is_Stop','HEAD_POS','CHILDREN_LEN'],axis=1)
  encoder=LabelEncoder()
  encoder.fit(df['Section'])
  df['Section_enc']=encoder.transform(df['Section'])
  encoder.fit(df['POS'])
  df['POS_enc']=encoder.transform(df['POS'])
  encoder.fit(df['Label'])
  df['Label_enc']=encoder.transform(df['Label'])
  encoder.fit(df['TAG'])
  df['TAG_enc']=encoder.transform(df['TAG'])
  encoder.fit(df['DEP'])
  df['DEP_enc']=encoder.transform(df['DEP'])
  encoder.fit(df['Shape'])
  df['Shape_enc']=encoder.transform(df['Shape'])
  encoder.fit(df['Is_Alpha'])
  df['Is_Alpha_enc']=encoder.transform(df['Is_Alpha'])
  encoder.fit(df['Is_Stop'])
  df['Is_Stop_enc']=encoder.transform(df['Is_Stop'])
  encoder.fit(df['HEAD_POS'])
  df['HEAD_POS_enc']=encoder.transform(df['HEAD_POS'])
  df=df.drop(['Section','POS','Label','TAG','DEP','Shape','Is_Alpha','Is_Stop','HEAD_POS'],axis=1)


PCA

In [3]:
class Preprocessing(object):
    @staticmethod
    def Feature_visualization(df):
        fig, axes = plt.subplots(math.ceil(len(df["feature_name"])/2),2,figsize=(18,12))
        negative = np.array([df["data"][i] for i in range(len(df["target"])) if df["target"][i] == 0])
        positive = np.array([df["data"][i] for i in range(len(df["target"])) if df["target"][i] == 1])

        ax = axes.ravel()
        df["data"] = np.array(df["data"])
        for i in range(len(df["feature_name"])):
            _, bins = np.histogram(df["data"][:,i],bins = 50)
            ax[i].hist(negative[:,i],bins=bins,alpha=.5)
            ax[i].hist(positive[:,i],bins=bins,alpha=.5)
            ax[i].set_title(df["feature_name"][i])
            ax[i].set_yticks(())
        ax[0].set_ylabel("Frequency")
        ax[0].legend(["Negative", "Positive"], loc="best")
        fig.tight_layout()
        return plt.show()

    @staticmethod
    def scale(df):
        scaler = StandardScaler()
        df["data"] = np.array(df["data"])
        scaler.fit(df["data"])
        X_scaled = scaler.transform(df["data"])
        return X_scaled

    @staticmethod
    def PCA_scatter(X_scaled,df):
        pca = PCA(n_components=2)
        pca.fit(X_scaled)

        X_pca = pca.transform(X_scaled)
        plt.figure(figsize=(18,12))
        mglearn.discrete_scatter(X_pca[:, 0], X_pca[:,1],df["target"])
        plt.legend(["negative","positive"])
        plt.gca().set_aspect("equal")
        plt.xlabel("First principal component")
        plt.ylabel("Second principal component")
        return plt.show()

    @staticmethod
    def PCA_heatmap(X_scaled,df):
        pca = PCA(n_components=2)
        pca.fit(X_scaled)
        plt.matshow(pca.components_,cmap="viridis")
        plt.yticks([0,1],["First component","Second component"])
        plt.colorbar()
        plt.xticks(range(len(df["feature_name"])),df["feature_name"], rotation=60, ha="left")
        plt.xlabel("Feature")
        plt.ylabel("Principal components")
        return plt.show()

    @staticmethod
    def TSNE_scatter(df):
        tsne = TSNE(random_state=42)
        face_tsne = tsne.fit_transform(df["data"])
        plt.figure(figsize=(18,12))
        plt.xlim(face_tsne[:,0].min(), face_tsne[:,0].max()+1)
        plt.ylim(face_tsne[:,1].min(), face_tsne[:,1].max()+1)
        for i in range(len(df["data"])):
            if df["target"][i] == 1: plt.text(face_tsne[i,0],face_tsne[i,1],
                        str("1"),color="red", fontdict= {"weight":"bold", "size":15})
            else:plt.text(face_tsne[i,0],face_tsne[i,1],
                        str("0"),color='blue', fontdict= {"weight":"bold", "size":15})
        plt.xlabel("t_SNE feature 0")
        plt.ylabel("t_SNE feature 1")
        return plt.show()

